<a href="https://colab.research.google.com/github/obeabi/ProjectPortfolio/blob/master/TableOperations_MySql_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install mysql-connector-python

In [8]:
# Import the modules
from flask import Flask, request, jsonify
import logging
import mysql.connector

In [ ]:
# Create flask object app
app = Flask(__name__)

# Configure the logging module
logging.basicConfig(filename="app.log", level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

In [ ]:
# Define the parameters for the databse connection
hostname = "localhost" # Change this to your hostname

password = "password" # Change this to your password

table_name = "employees" # Change this to your table name

database = "my_database" # Change this to your database name

table_fields = ["id", "name", "salary", "department"] # Change this to your table fields

data_types = ["INT PRIMARY KEY", "VARCHAR(255)", "FLOAT", "VARCHAR(255)"] # Change this to your data types for each field

In [7]:
# create a function that connects to the datbase and returns a cursor object

def connect_mysql_db(database):
  try:
      db = mysql.connector.connect(
          host = hostname,
          user = "root",
          password = password,
          database = database  # change this to your databse name
      )

      cursor = db.cursor()
      return db, cursor
  except mysql.connector.Error as e:
    logging.error(f"Failed to connect to database {database} : {e}")
    return None, None


In [ ]:
# Create a route that creates a table on the databse using the parameters
@app.route("/Table_operation/Mysql/create_table", methods = ["GET"])
def create_mysql_table():

  # connect to the databse
  db, cursor = connect_mysql_db(database)
  if db is None or cursor is None:
    return jsonify({"message" : f"Failed to connect to database {database} ."})

  # create sql statement that creates a table with specifief name, fields and data types
  sql = f"CREATE TABLE IF NOT EXISTS {table_name}("
  for i in range(len(table_fields)):
      sql += f"{table_fields[i]} {data_types[i]}"
      if i < len(table_fields) - 1:
        sql += ", "
  sql += ")"

  # Execute the SQL Statement and commit the changes
  try:
    cursor.execute(sql)
    db.commit()
    logging.info(f"Table {table_name} created successfully.")
    #Close the connection and return a success message
    cursor.close()
    db.close()
    return jsonify({"message" : f"Table {table_name} created succesfully ."})
  except mysql.connector.Error as e:
    logging.error(f"Failed to create table {table_name} : {e}")
    return jsonify({"message" : f"Failed to create table {table_name} ."})

In [ ]:
# Create a route that inserts a row of data into the table using the parameters

@app.route("/Table_operation/Mysql/insert_row", methods = ["POST"])
def insert_data_mysql():
  # Get the data from the request body as a JSON object
  data = request.get_json()
  # Validate the data and check if it has the same number and names of fields as the table
  if data is None or len(data) != len(table_fields) or any(key not in table_fields for key in data.keys()):
    return jsonify({"message" : f"invalid data. Please provide a JSON object with \
    {len(table_fields)} fields : {','.join(table_fields)}."})
  # Connect to the database
  db,cursor = connect_mysql_db(database)
  if db is None or cursor is None:
        return jsonify({"message": f"Failed to connect to database {database}."})

  # Create a SQL statement that inserts a row of data into the table with the specified values
  sql = f"INSERT INTO {table_name} ({', '.join(table_fields)}) VALUES ({', '.join(['%s'] * len(table_fields))})"
  values = [data[key] for key in table_fields]
  # Execute the SQL statement and commit the changes
  try:
        cursor.execute(sql, values)
        db.commit()
        logging.info(f"Data inserted successfully: {data}")
        # Close the connection and return a success message
        cursor.close()
        db.close()
        return jsonify({"message": f"Data inserted successfully: {data}"})
  except mysql.connector.Error as e:
        logging.error(f"Failed to insert data: {data}: {e}")
        return jsonify({"message": f"Failed to insert data: {data}."})


In [ ]:
# Create a route that updates row(s) in a table on the database
@app.route("/Table_operation/Mysql/update_table", methods = ["PUT"])
def update_mysql_table():
    # Get the data from the request body as a JSON object
    data = request.get_json()
    # Validate the data and check if it has a valid condition and at least one field to update
    if data is None or "condition" not in data or len(data) < 2 or any(key not in table_fields for key in data.keys() if key != "condition"):
        return jsonify({"message": f"Invalid data. Please provide a JSON object with a valid condition and at least one field to update from: {', '.join(table_fields)}."})
    # Connect to the database
    db, cursor = connect_mysql_db(database)
    if db is None or cursor is None:
        return jsonify({"message": f"Failed to connect to database {database}."})
    # Check if the table name exists in the database
    sql = f"SHOW TABLES LIKE '{table_name}'"
    cursor.execute(sql)
    result = cursor.fetchone()
    if result is None:
        # If the table name does not exist, do nothing and return a message
        logging.info(f"Table {table_name} does not exist.")
        return jsonify({"message": f"Table {table_name} does not exist."})
    else:
        # If the table name exists, create a SQL statement that updates rows in the table with the specified values and condition
        sql = f"UPDATE {table_name} SET "
        values = []
        for key, value in data.items():
            if key != "condition":
                sql += f"{key} = %s, "
                values.append(value)
        sql = sql[:-2] # Remove the last comma and space
        sql += f" WHERE {data['condition']}"
        # Execute the SQL statement and commit the changes
        try:
            cursor.execute(sql, values)
            db.commit()
            logging.info(f"Data updated successfully: {data}")
            # Close the connection and return a success message
            cursor.close()
            db.close()
            return jsonify({"message": f"Data updated successfully: {data}"})
        except mysql.connector.Error as e:
            logging.error(f"Failed to update data: {data}: {e}")
            return jsonify({"message": f"Failed to update data: {data}."})

An example for the input JSON for the update_data API is:

{
    "condition": "id = 1",
    "name": "Alice",
    "salary": 5000.0
}

In [ ]:
# Run the app
if __name__ == "__main__":
    try:
        app.run(debug=True)
        logging.info("App running successfully.")
    except Exception as e:
        logging.error(f"App failed: {e}")